<a href="https://colab.research.google.com/github/caumannerman/ncm_project/blob/master/%EC%B1%942.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pykrx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.5/125.5 KB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.4/52.4 KB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 KB 14.3 MB/s eta 0:00:00


In [3]:
from pykrx import stock
import pandas as pd
import requests
import time
from datetime import datetime
 
 
def per(date):
    codes = stock.get_market_ticker_list(date, market='ALL')
    corp = []
 
    for code in codes:
        name = stock.get_market_ticker_name(code)
        corp.append([code, name])
 
    df = pd.DataFrame(data=corp, columns=['종목코드', '종목명'])
    df = df.set_index('종목코드')
    df_f = stock.get_market_fundamental_by_ticker(date, market='ALL')
    df = pd.merge(df, df_f, left_index=True, right_index=True)
 
    return df
 
 
def get_roa(code, year):
      url = 'https://comp.fnguide.com/SVO2/ASP/SVD_FinanceRatio.asp?pGB=1&gicode=A'+ code \
            +'&cID=&MenuYn=Y&ReportGB=&NewMenuID=104&stkGb=701'
 
 
      fr_page = requests.get(url)
      fr_tables = pd.read_html(fr_page.text)
 
      temp_df = fr_tables[0]
      temp_df = temp_df.set_index(temp_df.columns[0]) #가져온 칼럼의 첫번째 열을 인덱스로 지정하기
      temp_df = temp_df.loc[['ROA계산에 참여한 계정 펼치기']] #'ROA계산에 참여한 계정 펼치기' 행만 가져오기
      temp_df.index = ['ROA'] #인덱스 이름을 'ROA'로 수정하기
 
      temp_df = pd.DataFrame({code:temp_df[year]}) #가져올 시점에 따라 데이터프레임 만들기
      temp_df = temp_df.transpose() #행과 열의 위치를 바꾸기
 
      return temp_df
 
 
def make_magic_df(year, date):
 
      # #KRX 데이터 "종목코드", "종목명", "PER" 가져오기
      codes = per(date)
      codes['종목코드'] = codes.index
 
      # ROA를 가져와서 데이트프레임 만들기
      for num, code in enumerate(codes['종목코드']):
            try:
                  if num == 0:
                        roa_df = get_roa(code, year)
                  else:
                        roa_df = pd.concat([roa_df, get_roa(code, year)])
                  time.sleep(0.01)
            except:
                  pass
 
            print(len(codes['종목코드']) - num, codes['종목명'][num])
      # 종목코드 칼럼을 index로 바꾸기
      codes = codes.set_index('종목코드')
 
      # ROA, ROA 문자형 데이터를 숫자로 변경
      roa_df['ROA'] = pd.to_numeric(roa_df['ROA'], errors='coerce') #문자는 에러를 발생하므로 N/A로 변경
      roa_df = roa_df['ROA'].dropna()
 
      # 전체 마법의 공식 데이터 만들기
      total_df = pd.merge(codes, roa_df, left_index=True, right_index=True)
 
      # 값이 0인 행 제거
      total_df = total_df[total_df['PER'] != 0]
 
      # PER 순위매기기
      total_df['per_rank'] = total_df['PER'].rank()
      total_df['roa_rank'] = total_df['ROA'].rank(ascending=False)
      total_df['total'] = total_df['per_rank'] + total_df['roa_rank']
      total_df['total_rank'] = total_df['total'].rank()
      total_df = total_df.sort_values(by=['total_rank'])
 
      today = datetime.today().strftime('%y-%m-%d')
      total_df.to_excel('Magic(' + 'today' + ').xlsx')
 
      return total_df
 
 
 
 
year = '2022/09'
date = '20230106'
 
print(make_magic_df(year, date))

2509 3S
2508 AJ네트웍스
2507 AK홀딩스
2506 APS홀딩스
2505 AP시스템
2504 AP위성
2503 BGF
2502 BGF리테일
2501 BNK금융지주
2500 BYC
2499 BYC우
2498 CBI
2497 CJ
2496 CJ CGV
2495 CJ ENM
2494 CJ 바이오사이언스
2493 CJ4우(전환)
2492 CJ대한통운
2491 CJ씨푸드
2490 CJ씨푸드1우
2489 CJ우
2488 CJ제일제당
2487 CJ제일제당 우
2486 CJ프레시웨이
2485 CMG제약
2484 CNH
2483 CNT85
2482 CS
2481 CSA 코스믹
2480 CS홀딩스
2479 DB
2478 DB금융스팩10호
2477 DB금융스팩8호
2476 DB금융스팩9호
2475 DB금융투자
2474 DB손해보험
2473 DB하이텍
2472 DB하이텍1우
2471 DGB금융지주
2470 DI동일
2469 DL
2468 DL건설
2467 DL우
2466 DL이앤씨
2465 DL이앤씨2우(전환)
2464 DL이앤씨우
2463 DMS
2462 DN오토모티브
2461 DRB동일
2460 DSC인베스트먼트
2459 DSR
2458 DSR제강
2457 E1
2456 EDGC
2455 EG
2454 ES큐브
2453 EV수성
2452 F&F
2451 F&F홀딩스
2450 FSN
2449 GH신소재
2448 GKL
2447 GS
2446 GST
2445 GS건설
2444 GS글로벌
2443 GS리테일
2442 GS우
2441 HB솔루션
2440 HB테크놀러지
2439 HDC
2438 HDC랩스
2437 HDC현대EP
2436 HDC현대산업개발
2435 HD현대
2434 HJ중공업
2433 HK이노엔
2432 HL D&I
2431 HLB
2430 HLB글로벌
2429 HLB생명과학
2428 HLB제약
2427 HLB테라퓨틱스
2426 HL만도
2425 HL홀딩스
2424 HMM
2423 HPSP
2422 HRS
2421 HSD엔진
2420 IBKS제13호스팩
241

# 새 섹션